<a href="https://colab.research.google.com/github/clayajohnson/gpt-2/blob/master/Finetune_GPT_2_on_JRE_transcripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetune GPT-2 on Joe Rogan Experience transcripts

In [1]:
!git clone https://github.com/clayajohnson/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 395 (delta 9), reused 16 (delta 6), pack-reused 375
Receiving objects: 100% (395/395), 4.72 MiB | 3.35 MiB/s, done.
Resolving deltas: 100% (214/214), done.


In [7]:
!nvidia-smi


Thu Jul  9 10:46:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [19]:
# Move into gpt-2 folder
%cd /content/gpt-2
# !pip3 install -r requirements.txt

/content/gpt-2


In [1]:
!pip install tensorflow-gpu==1.15.0
!pip install 'tensorflow-estimator<1.15.0rc0,>=1.14.0rc0' --force-reinstall

     |████████████████████████████████| 411.5MB 42kB/s 
     |████████████████████████████████| 512kB 43.2MB/s 
     |████████████████████████████████| 3.8MB 48.6MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=5c1c6799f42efe24d01f205de56fa6cc3ee44f2d69a0f51e15d66308eb89aa88
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninst

In [2]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-*/7fa2af80.pub
!apt-get update
!apt-get install cuda-9-0
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda-9.0/lib64/

--2020-07-05 21:40:45--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.16.29
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.16.29|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb?JmZrKxl5gbrHfBYkGJ7cPrHzN-K0635sVZNadZC8UQDAgEdAlV3Va_g-FYIOMObgybYrxZ_u9yYl7dsOmPeqmllE8wN9T2g7qAc0a3TiAaipIvFE14YPm2lQMmgdOS1zIKWvl-Dv_Gp-TkQ2KvMfaPpiUcN5X4v8SFdWVF0qFOrlhsa-1dEIMLc0jesnwVS6c52k1Vkzb-4955eEykux [following]
--2020-07-05 21:40:45--  https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb?JmZrKxl5gbrHfBYkGJ7cPrHzN-K0635sVZNadZC8UQDAgEdAlV3Va_g-FYIOMObgybYrxZ_u9yYl7dsOmPeqmllE8wN9T2g7qAc0a3TiAaipIvFE14

In [4]:
%cd gpt-2/

/content/gpt-2


In [20]:
!python3 download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 654kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 47.6Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 1.22Mit/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:17, 81.1Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 8.98Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 60.5Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 56.6Mit/s]                                                       


In [21]:
!PYTHONPATH=src ./train.py --dataset jre_dataset.txt --model_name 345M


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




2020-07-05 22:26:36.482181: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2020-07-05 22:26:36.486649: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000175000 Hz
2020-07-05 22:26:36.486846: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1dc5640 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-07-05 22:26:36.486874: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-07-05 22:26:3

In [7]:
%cp checkpoint/run1/* models/117M/

In [22]:
!PYTHONPATH=src ./src/interactive_conditional_samples.py --run_name run1



2020-07-05 22:31:36.360269: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-05 22:31:36.383150: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-05 22:31:36.384744: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:00:04.0
2020-07-05 22:31:36.391303: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-07-05 22:31:36.400778: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-07-05 22:31:36.407514: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library li